In [392]:
import pandas as pd

passengers = pd.read_csv("Data/train.csv")
passengers.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [393]:
from sklearn.model_selection import train_test_split

X = passengers.drop(columns=["Survived"])
Y = passengers["Survived"]
x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=0.1)

In [394]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer


class DropColumnTranformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.drop(columns=self.columns, axis=1, errors="ignore")
    
    def get_feature_names_out(self, input_features=None):
        return None
    
    
    
class SumTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column1, column2):
        self.column1 = column1
        self.column2 = column2
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return (X[self.column1] + X[self.column2]).to_frame()
    
    def get_feature_names_out(self, input_features=None):
        return ["Family"]
    
    
class AgeGroupTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column="Age"):
        self.age_bins = [0, 3, 12, 20, 40, 60, 200]
        self.age_labels = ["toddler", "child", "teen", "young-adult", "middle-adult", "senior"]
        self.column = column
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=[self.column])
        X["AgeGroup"] = pd.cut(X[self.column], bins=self.age_bins, labels=self.age_labels)
        return X[["AgeGroup"]].astype(str).to_numpy()
    
    def get_feature_names_out(self, input_features=None):
        return ["AgeGroup"]
        
        
class DeckExctractorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X, columns=[self.column])
        X["Deck"] = X[self.column].str[0]
        X["Deck"].fillna("N", inplace=True)
        return X[["Deck"]].to_numpy()
    
    def get_feature_names_out(self, names=None):
        return ["Deck"]
    
    
class PassthroughTransformer(BaseEstimator):
  def fit(self, X, y = None):
    self.cols = X.columns
    return self

  def transform(self, X, y = None):
    self.cols = X.columns
    return X.values

  def get_feature_names_out(self, names=None):
    return self.cols
        
        
log_transformer = FunctionTransformer(np.log, feature_names_out="one-to-one")

In [395]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

binary_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"),
                                OneHotEncoder())

age_group_pipeline = make_pipeline(SimpleImputer(strategy="mean"),
                                   AgeGroupTransformer(),
                                   OneHotEncoder(handle_unknown="ignore"))

family_pipeline = make_pipeline(SumTransformer("SibSp", "Parch"))

log_pipeline = make_pipeline(SimpleImputer(missing_values=0, strategy="median"),
                             SimpleImputer(strategy="median"),
                             log_transformer,
                             StandardScaler())

deck_pipeline = make_pipeline(DeckExctractorTransformer("Cabin"),
                              OneHotEncoder(handle_unknown="ignore"))

num_pipeline = make_pipeline(SimpleImputer(strategy="mean"),
                             StandardScaler())

cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"),
                             OneHotEncoder(handle_unknown="ignore"))

drop_pipeline = make_pipeline(DropColumnTranformer(["Name", "Ticket", "SibSp", "Parch", "PassengerId"]))

In [396]:
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer([
    ("sex", binary_pipeline, ["Sex"]),
    ("age_group", age_group_pipeline, ["Age"]),
    ("family", family_pipeline, ["SibSp", "Parch"]),
    ("fare", log_pipeline, ["Fare"]),
    ("deck", deck_pipeline, ["Cabin"]),
    ("drop", drop_pipeline, ["Name", "Ticket", "SibSp", "Parch", "PassengerId"]),
    ("cat", cat_pipeline, ["Embarked"]),
    ("num", num_pipeline, ["Age"])
], remainder=PassthroughTransformer())

In [397]:
x_train_transformed = preprocessing.fit_transform(x_train)
feature_names = preprocessing.get_feature_names_out()

pd.DataFrame(x_train_transformed, columns=feature_names)

,sex__Sex_female,sex__Sex_male,age_group__AgeGroup_child,age_group__AgeGroup_middle-adult,age_group__AgeGroup_senior,age_group__AgeGroup_teen,age_group__AgeGroup_toddler,age_group__AgeGroup_young-adult,family__Family,fare__Fare,...,deck__Deck_E,deck__Deck_F,deck__Deck_G,deck__Deck_N,deck__Deck_T,cat__Embarked_C,cat__Embarked_Q,cat__Embarked_S,num__Age,remainder__Pclass
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.519194,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.000000,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.568805,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.061398,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.516092,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.798151,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.299875,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.194439,3.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.582953,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.139538,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.645878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.0
797,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.296492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.0
798,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.350378,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,1.0
799,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.952152,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.233510,3.0
